In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import psycopg2
import streamlit as st

In [2]:
#Connect to local postgres DB
conn = psycopg2.connect(**st.secrets["postgres"])
cur = conn.cursor()

In [3]:
def run_query(query):
    with conn.cursor() as cur:
        cur.execute(query)
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        cur.close()
        return pd.DataFrame(results, columns=columns)

#### SELECT ALL

In [4]:
select_all_query = """SELECT * FROM oil_fund;"""    
df = run_query(select_all_query)
df

,region,country,name,industry,market_value,category,year
0,Oceania,Australia,Amcor Ltd,Industrials,1779853,Equity,1998
1,Oceania,Australia,AMP Ltd,Financials,8662515,Equity,1998
2,Oceania,Australia,Ashton Mining,Basic Materials,75850,Equity,1998
3,Oceania,Australia,Australian Foundation Investment Co Ltd,Financials,793252,Equity,1998
4,Oceania,Australia,Australian Gas Light Company,Utilities,1509918,Equity,1998
...,...,...,...,...,...,...,...
189336,North America,United States,WPC Eurobond BV,Corporate Bonds,79539339,Fixed Income,2022
189337,North America,United States,Wyeth LLC,Corporate Bonds,37798334,Fixed Income,2022
189338,North America,United States,XLIT Ltd,Corporate Bonds,13322191,Fixed Income,2022
189339,North America,United States,Zimmer Biomet Holdings Inc,Corporate Bonds,40329494,Fixed Income,2022


## Questions To Answer:

- What is the proportion between equity and fixed income over time?
- What is the proportion between industries over time?
- What is the proportion between regions over time?
- What is the YoY growth rate % of the fund (this is not the return % since I do not know what the funds inflows or average holding costs are)
- What is the average YoY growth rate for the various industries? What is the standard deviation?
- What is the YoY growth rate % for regions?

### Equity and Fixed Income Proportions

In [5]:
equity_fi_query = """SELECT year,
ROUND((SUM(CASE WHEN category = 'Equity' THEN market_value ELSE 0 END) / NULLIF(SUM(market_value),0)) * 100, 2) AS "Equity Percentage",
ROUND((SUM(CASE WHEN category = 'Fixed Income' THEN market_value ELSE 0 END) / NULLIF(SUM(market_value),0)) * 100, 2) AS "Fixed Income Percentage"
FROM oil_fund
GROUP BY year;
"""

equity_fi_df = run_query(equity_fi_query)
equity_fi_df

,year,Equity Percentage,Fixed Income Percentage
0,1998,39.66,60.34
1,1999,41.87,58.13
2,2000,38.13,61.87
3,2001,40.78,59.22
4,2002,36.74,63.26
5,2003,42.91,57.09
6,2004,39.61,60.39
7,2005,46.47,53.53
8,2006,38.18,61.82
9,2007,45.81,54.19


### Industry Proportions Over Time

In [6]:
industry_prop = """SELECT year, industry as "Industry",
ROUND(sum(market_value) / sum(sum(market_value)) OVER (PARTITION BY year) * 100.0,2) as "Proportion of Fund"
FROM oil_fund
GROUP BY year, industry;
"""

industry_prop_df = run_query(industry_prop)
industry_prop_df

,year,Industry,Proportion of Fund
0,1998,Basic Materials,1.67
1,1998,Consumer Discretionary,4.69
2,1998,Consumer Staples,4.68
3,1998,Corporate Bonds,1.28
4,1998,Energy,2.23
...,...,...,...
370,2022,Securitized Bonds,1.27
371,2022,Technology,12.19
372,2022,Telecommunications,2.13
373,2022,Treasuries,19.42


### Region Proportions Over Time